In [ ]:
!wget -q -O - ipv4.icanhazip.com



In [ ]:
!pip install streamlit -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.2 MB/s eta 0:00:00


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼

⠴⠦⠧⠇⠏⠋
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.145.181.245:8501

your url is: https://solid-buses-marry.loca.lt
/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:27163 (check your firewall settings)
    at Socket.<anonymous> (/root/.npm/_npx/75ac80b86e83d4a2/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙  Stopping...


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import math
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import logging

logging.getLogger('prophet').setLevel(logging.WARNING)

print("--- 1. Data Loading and Preprocessing ---")
df = pd.read_csv('AAL.csv')

df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df.sort_values(by='Date', inplace=True)

print("First 5 rows of the dataset:")
print(df.head())
print("\nLast 5 rows of the dataset:")
print(df.tail())
print("\nMissing values per column:")
print(df.isna().sum())
print("\nDataFrame Info:")
df.info()
print("\nDataFrame Columns:")
print(df.columns)
print("\nDataFrame Description:")
print(df.describe())

print("\n--- 2. LSTM Model Implementation ---")
features = ['Open', 'High', 'Low', 'Volume', 'Adj Close']
target = 'Close'
X = df[features].values
y = df[target].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train_lstm, X_test_lstm, y_train_lstm, y_test_lstm = train_test_split(X_scaled, y, test_size=0.2, random_state=42, shuffle=False)
X_train_lstm = X_train_lstm.reshape(X_train_lstm.shape[0], 1, X_train_lstm.shape[1])
X_test_lstm = X_test_lstm.reshape(X_test_lstm.shape[0], 1, X_test_lstm.shape[1])

lstm_model = Sequential()
lstm_model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(units=1))
lstm_model.compile(optimizer='adam', loss='mean_squared_error')

print("Training LSTM model...")
lstm_model.fit(X_train_lstm, y_train_lstm, epochs=100, batch_size=64, verbose=0)
lstm_predictions = lstm_model.predict(X_test_lstm).flatten()

mse_lstm = mean_squared_error(y_test_lstm, lstm_predictions)
mae_lstm = mean_absolute_error(y_test_lstm, lstm_predictions)
rmse_lstm = math.sqrt(mse_lstm)
r2_lstm = r2_score(y_test_lstm, lstm_predictions)

print("\nLSTM Model Evaluation:")
print(f"Mean Squared Error: {mse_lstm:.4f}")
print(f"Mean Absolute Error: {mae_lstm:.4f}")
print(f"Root Mean Squared Error: {rmse_lstm:.4f}")
print(f"R^2 Score: {r2_lstm:.4f}")

plt.figure(figsize=(12, 6))
test_dates_lstm = df.index[-len(y_test_lstm):]
plt.plot(test_dates_lstm, y_test_lstm, label='Actual Close Price')
plt.plot(test_dates_lstm, lstm_predictions, label='LSTM Predicted Close Price', color='red', linestyle='--')
plt.title('LSTM Model: Actual vs Predicted Close Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("lstm_plot.png")
plt.close()

print("\n--- 2.1. LSTM Future Predictions ---")
future_input_data = df[features].tail(10).values
future_input_scaled = scaler.transform(future_input_data)
future_input_reshaped = future_input_scaled.reshape(future_input_scaled.shape[0], 1, future_input_scaled.shape[1])
future_lstm_predictions = lstm_model.predict(future_input_reshaped).flatten()

print("LSTM Predicted Close Prices for the next 10 data points (based on last 10 feature sets):")
for i, price in enumerate(future_lstm_predictions):
    print(f"Day {i+1}: {price:.2f}")

print("\n--- 3. ARIMA Model Implementation ---")
y_train_arima = df[target].iloc[:len(y_train_lstm)]
y_test_arima = df[target].iloc[len(y_train_lstm):]

print("Training ARIMA model...")
arima_model = ARIMA(y_train_arima, order=(5,1,0))
arima_results = arima_model.fit()

arima_predictions_series = arima_results.forecast(steps=len(y_test_arima))
arima_predictions = arima_predictions_series.values

mse_arima = mean_squared_error(y_test_arima, arima_predictions)
mae_arima = mean_absolute_error(y_test_arima, arima_predictions)
rmse_arima = math.sqrt(mse_arima)
r2_arima = r2_score(y_test_arima, arima_predictions)

print("\nARIMA Model Evaluation:")
print(f"Mean Squared Error: {mse_arima:.4f}")
print(f"Mean Absolute Error: {mae_arima:.4f}")
print(f"Root Mean Squared Error: {rmse_arima:.4f}")
print(f"R^2 Score: {r2_arima:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(test_dates_lstm, y_test_arima, label='Actual Close Price')
plt.plot(test_dates_lstm, arima_predictions, label='ARIMA Predicted Close Price', color='green', linestyle='--')
plt.title('ARIMA Model: Actual vs Predicted Close Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("arima_plot.png")
plt.close()

print("\n--- 4. SARIMAX Model Implementation ---")
y_train_sarimax = df[target].iloc[:len(y_train_lstm)]
y_test_sarimax = df[target].iloc[len(y_train_lstm):]

print("Training SARIMAX model...")
sarimax_model = SARIMAX(y_train_sarimax, order=(1, 1, 1), seasonal_order=(1, 1, 1, 12))
sarimax_results = sarimax_model.fit(disp=False)

sarimax_predictions_series = sarimax_results.forecast(steps=len(y_test_sarimax))
sarimax_predictions = sarimax_predictions_series.values

mse_sarimax = mean_squared_error(y_test_sarimax, sarimax_predictions)
mae_sarimax = mean_absolute_error(y_test_sarimax, sarimax_predictions)
rmse_sarimax = math.sqrt(mse_sarimax)
r2_sarimax = r2_score(y_test_sarimax, sarimax_predictions)

print("\nSARIMAX Model Evaluation:")
print(f"Mean Squared Error: {mse_sarimax:.4f}")
print(f"Mean Absolute Error: {mae_sarimax:.4f}")
print(f"Root Mean Squared Error: {rmse_sarimax:.4f}")
print(f"R^2 Score: {r2_sarimax:.4f}")

plt.figure(figsize=(12, 6))
plt.plot(test_dates_lstm, y_test_sarimax, label='Actual Close Price')
plt.plot(test_dates_lstm, sarimax_predictions, label='SARIMAX Predicted Close Price', color='purple', linestyle='--')
plt.title('SARIMAX Model: Actual vs Predicted Close Price')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("sarimax_plot.png")
plt.close()

print("\n--- 5. Prophet Model Implementation ---")
df_prophet = df.reset_index()[['Date', 'Close']]
df_prophet.rename(columns={'Date': 'ds', 'Close': 'y'}, inplace=True)

train_size_prophet = int(len(df_prophet) * 0.8)
df_prophet_train = df_prophet.iloc[:train_size_prophet]
df_prophet_test = df_prophet.iloc[train_size_prophet:]

prophet_model = Prophet()
print("Training Prophet model...")
prophet_model.fit(df_prophet_train)

future_prophet = prophet_model.make_future_dataframe(periods=len(df_prophet_test), include_history=False)
forecast_prophet = prophet_model.predict(future_prophet)

y_pred_prophet = forecast_prophet['yhat'].values
y_test_prophet = df_prophet_test['y'].values

prophet_mse = mean_squared_error(y_test_prophet, y_pred_prophet)
prophet_mae = mean_absolute_error(y_test_prophet, y_pred_prophet)
prophet_rmse = math.sqrt(prophet_mse)
prophet_r2 = r2_score(y_test_prophet, y_pred_prophet)

print("\nProphet Model Evaluation (on Test Set):")
print(f"Mean Squared Error: {prophet_mse:.4f}")
print(f"Mean Absolute Error: {prophet_mae:.4f}")
print(f"Root Mean Squared Error: {prophet_rmse:.4f}")
print(f"R^2 Score: {prophet_r2:.4f}")

fig_prophet1 = prophet_model.plot(forecast_prophet)
plt.title("Prophet Forecast - Close Price (Test Set Highlighted)")
plt.xlabel("Date")
plt.ylabel("Close Price")
plt.axvline(x=df_prophet_train['ds'].iloc[-1], color='grey', linestyle='--', label='Train/Test Split')
plt.legend()
fig_prophet1.savefig("prophet_plot_forecast.png")
plt.close(fig_prophet1)

fig_prophet2 = prophet_model.plot_components(forecast_prophet)
fig_prophet2.savefig("prophet_plot_components.png")
plt.close(fig_prophet2)

plt.figure(figsize=(12, 6))
plt.plot(df_prophet_test['ds'], y_test_prophet, label='Actual Close Price')
plt.plot(df_prophet_test['ds'], y_pred_prophet, label='Prophet Predicted Close Price', color='orange', linestyle='--')
plt.title('Prophet Model: Actual vs Predicted Close Price (Test Set)')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("prophet_plot.png")
plt.close()

print("\n--- 6. Model Comparison ---")
results = pd.DataFrame({
    'Model': ['LSTM', 'ARIMA', 'SARIMAX', 'Prophet'],
    'MSE': [mse_lstm, mse_arima, mse_sarimax, prophet_mse],
    'MAE': [mae_lstm, mae_arima, mae_sarimax, prophet_mae],
    'RMSE': [rmse_lstm, rmse_arima, rmse_sarimax, prophet_rmse],
    'R^2 Score': [r2_lstm, r2_arima, r2_sarimax, prophet_r2]
})

print("--- Model Performance Comparison ---")
print(results.round(4))
results.to_csv("model_evaluation_summary.csv", index=False)

print("\n--- 7. Discussion and Conclusion ---")
print("Based on the evaluation metrics:")
print("\n--- Project Completed ---")
print("📄 model_evaluation_summary.csv saved!")
print("📈 Plots saved as PNG files!")


--- 1. Data Loading and Preprocessing ---
First 5 rows of the dataset:
                 Open       High        Low      Close  Adj Close   Volume
Date                                                                      
2005-09-27  21.049999  21.400000  19.100000  19.299999  18.194912   961200
2005-09-28  19.299999  20.530001  19.200001  20.500000  19.326204  5747900
2005-09-29  20.400000  20.580000  20.100000  20.209999  19.052807  1078200
2005-09-30  20.260000  21.049999  20.180000  21.010000  19.806997  3123300
2005-10-03  20.900000  21.750000  20.900000  21.500000  20.268946  1057900

Last 5 rows of the dataset:
                 Open       High    Low      Close  Adj Close    Volume
Date                                                                   
2022-03-18  16.350000  16.840000  16.16  16.750000  16.750000  38979100
2022-03-21  16.500000  16.540001  15.90  16.090000  16.090000  45852600
2022-03-22  16.200001  16.760000  16.09  16.559999  16.559999  40143400
2022-03-23  16.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Training LSTM model...


In [ ]:
from google.colab import files
files.download('model_evaluation_summary.csv')
files.download('lstm_plot.png')
files.download('arima_plot.png')
files.download('sarimax_plot.png')
files.download('prophet_plot.png')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>